# How to Convert XGBoost to ONNX

The following tutorial is a brief example of how to convert a [XGBoost](https://xgboost.readthedocs.io/en/stable/index.html) ML model to the [ONNX](https://onnx.ai/ ) standard.  This allows organizations that have trained XGBoost models to convert them and use them with Wallaroo.

This tutorial assumes that you have a Wallaroo instance and are running this Notebook from the Wallaroo Jupyter Hub service.

This tutorial provides the following:

* `isolet_xgboost_model.pickle`: a trained tree-based classifier XGBoost model with 617 columns.

## Conversion Process

### Libraries

The first step is to import our libraries we will be using.

In [1]:
import onnx
from onnxmltools.convert import convert_xgboost

from skl2onnx.common.data_types import FloatTensorType, DoubleTensorType

import pickle

## Load the XGBoost Model

Next we will load our model that has been saved in the `pickle` format and unpickle it.

In [2]:
# load the xgboost model
with open("isolet_xgboost_model.pickle", "rb") as f:
    xgboost_model = pickle.load(f)

## Conversion Inputs

The `convert_xgboost` method has the following format and requires the following inputs:

```
convert_xgboost({XGBoost Model}, 
                {XGBoost Model Type},
                [
                    ('input', 
                    {Tensor Data Type}([None, {ncols}]))
                ],
                target_opset={TARGET_OPSET})
```
    
1. **XGBoost Model**:  The XGBoost Model to convert.
1. **XGBoost Model Type**: The type of XGBoost model.  In this example is it a `tree-based classifier`.
1. **Tensor Data Type**:  Either `FloatTensorType` or `DoubleTensorType` from the `skl2onnx.common.data_types` library.
1. **ncols**:  Number of columns in the model.
1. **TARGET_OPSET**:  The target opset which can be derived in code showed below.

### Set the inputs

We already know some information about our model, such as the XGBoost model type and number of columns.  We'll set those variables below so we can use them in our `convert_xgboost` function.

In [3]:
# set the number of columns
ncols = 617

# derive the opset value

from onnx.defs import onnx_opset_version
from onnxconverter_common.onnx_ex import DEFAULT_OPSET_NUMBER
TARGET_OPSET = min(DEFAULT_OPSET_NUMBER, onnx_opset_version())
TARGET_OPSET

13

## Convert the Model

With all of our data in place we can now convert our XBBoost model to ONNX using the `convert_xgboost` method.

In [4]:
onnx_model_converted = convert_xgboost(xgboost_model, 'tree-based classifier',
                             [('input', FloatTensorType([None, ncols]))],
                             target_opset=TARGET_OPSET)

## Save the Model

With the model converted to ONNX, we can now save it and use it in a Wallaroo pipeline.

In [5]:
onnx.save_model(onnx_model_converted, "isolet_tree-based-classifier.onnx")